In [1]:
import re
import nltk

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
def document_to_wordlist( review, remove_stopwords=False ):
	review_text = BeautifulSoup(review).get_text()
	review_text = re.sub("[^a-zA-Z]"," ", review_text)
	words = review_text.lower().split()
	if remove_stopwords:
		stops = set(stopwords.words("english"))
		words = [w for w in words if not w in stops]
	return(words)

In [8]:
import itertools
from collections import Counter
raw_sentences = ["this is anirudh","hello who is this","anirudh is speacking","this is anirudh"]
wordlist = []
for each in raw_sentences:
    wordlist+=document_to_wordlist(each)
c = Counter(wordlist)
wordcloud = []
for each in c:
    temp = {}
    temp['text'] = each
    temp['size'] = c[each]
    wordcloud.append(temp)

In [10]:

tfidf = TfidfVectorizer().fit_transform(raw_sentences)
matrix = (tfidf * tfidf.T).A
print matrix
force = {}
force["nodes"] = []
force["links"] = [] 
for each in raw_sentences:
    temp={}
    temp["name"] = each
    force["nodes"].append(temp)
for ((i,_),(j,_)) in itertools.combinations(enumerate(raw_sentences), 2):
    temp = {}
    temp["source"] = i
    temp["target"] = j
    temp["value"] = matrix[i][j]
    force["links"].append(temp)
force


[[ 1.          0.3982426   0.50300644  1.        ]
 [ 0.3982426   1.          0.12835482  0.3982426 ]
 [ 0.50300644  0.12835482  1.          0.50300644]
 [ 1.          0.3982426   0.50300644  1.        ]]


{'links': [{'source': 0, 'target': 1, 'value': 0.3982425979778621},
  {'source': 0, 'target': 2, 'value': 0.50300644050587495},
  {'source': 0, 'target': 3, 'value': 0.99999999999999989},
  {'source': 1, 'target': 2, 'value': 0.12835481850733338},
  {'source': 1, 'target': 3, 'value': 0.3982425979778621},
  {'source': 2, 'target': 3, 'value': 0.50300644050587495}],
 'nodes': [{'name': 'this is anirudh'},
  {'name': 'hello who is this'},
  {'name': 'anirudh is speacking'},
  {'name': 'this is anirudh'}]}

In [4]:
import os
import numpy as np
def load_word2vec(dir):
    load_word2vec
    word2vec = {}
    for path in os.listdir(dir):
        iword2vec = {}
        #load the word2vec features.
        with open(os.path.join(dir,path), 'r') as fin:
            if path == 'vectors0.txt':
                next(fin) #skip information on first line
            for line in fin:
                items = line.replace('\r','').replace('\n','').split(' ')
                if len(items) < 10: continue
                word = items[0]
                vect = np.array([float(i) for i in items[1:] if len(i) > 1])
                iword2vec[word] = vect

        word2vec.update(iword2vec)
        
    return word2vec

In [5]:
word_vector= load_word2vec("static\\vectors")

In [6]:
def document_to_wordlist( review, remove_stopwords=False):
	'''
		Takes a string and converts it to wordlist(list)
	'''
	review_text = BeautifulSoup(review).get_text()
	review_text = re.sub("[^a-zA-Z]"," ", review_text)
	words = review_text.lower().split()
	if remove_stopwords:
		stops = set(stopwords.words("english"))
		words = [w for w in words if not w in stops]
	return(words)

In [20]:
from sklearn.metrics.pairwise import cosine_similarity
raw_sentences = ['Hello, This is anirudh. I am studying in MSIT.','hELLO, THIS IS SNEHA STUDYING msit.']
matrix = []
matrix = []
for each in range(len(raw_sentences)):
    li=[]
    for each1 in range(len(raw_sentences)):
        li.append(0)
    matrix.append(li)
for i in range(0,len(raw_sentences)):
    for j in range(0,len(raw_sentences)):
        sen1 = raw_sentences[i]
        sen2 = raw_sentences[j]
        sen1_words = document_to_wordlist(sen1)
        sen2_words = document_to_wordlist(sen2)
        sen1_vectors = []
        for each in sen1_words:
            if each in word_vector:
                sen1_vectors.append(word_vector[each])
        sen1_vector = np.array(sen1_vectors).sum(axis=0)
        sen2_vectors = []
        for each in sen2_words:
            if each in word_vector:
                sen2_vectors.append(word_vector[each])
        sen2_vector = np.array(sen2_vectors).sum(axis=0)
        
        matrix[i][j] = cosine_similarity(sen1_vector, sen2_vector)[0][0]
print matrix


[[1.0000000000000002, 0.83357583365549581], [0.83357583365549581, 1.0000000000000002]]


In [19]:
force = {}
import itertools
import json
from collections import Counter
force["nodes"] = []
force["links"] = [] 
for each in raw_sentences:
    temp={}
    temp["name"] = each
    force["nodes"].append(temp)
for ((i,_),(j,_)) in itertools.combinations(enumerate(raw_sentences), 2):
    temp = {}
    temp["source"] = i
    temp["target"] = j
    temp["value"] = matrix[i][j]
    force["links"].append(temp)
graph = json.dumps(force)
wordlist = []
for each in raw_sentences:
    wordlist+=document_to_wordlist(each)
c = Counter(wordlist)
wordcloud = []
for each in c:
    temp = {}
    temp["text"] = each
    temp["size"] = c[each]*20
    wordcloud.append(temp)
wordcloud = json.dumps(wordcloud)
print graph

{"nodes": [{"name": "Hello, This is anirudh. I am studying in MSIT."}, {"name": "hELLO, THIS IS SNEHA STUDYING msit."}], "links": [{"source": 0, "target": 1, "value": 0.83357583365549581}]}


In [10]:

print matrix


[[1.0000000000000002, 0.83357583365549581], [0.83357583365549581, 1.0000000000000002]]


In [22]:
texts = [] 
from gensim import corpora, models, similarities
raw_sentences = ['Hello, This is anirudh', 'I am studying in MSIT','hELLO, THIS IS SNEHA','i STUDY AT msit.']
matrix = np.zeros(shape=(len(raw_sentences), len(raw_sentences)))
print matrix
for each in raw_sentences:
    texts.append(document_to_wordlist(each))
print texts

dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
lsii = models.LsiModel(corpus)

matrix = np.zeros(shape=(len(raw_sentences), len(raw_sentences)))

for i in range(len(raw_sentences)):
    vec = corpus[i]
    doc = raw_sentences[i]

    vec_bow = dictionary.doc2bow(doc.lower().split())
    vec_lsi = lsii[vec_bow]  # convert the query to LSI space

    index = similarities.MatrixSimilarity(lsii[corpus])
    sims = index[vec_lsi]  # perform a similarity query against the corpus
    cosine = list(enumerate(sims))
    for j in range(len(raw_sentences)):
        matrix[i][j] = cosine[j][1]
print matrix

[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]
[[u'hello', u'this', u'is', u'anirudh'], [u'i', u'am', u'studying', u'in', u'msit'], [u'hello', u'this', u'is', u'sneha'], [u'i', u'study', u'at', u'msit']]
[[ 0.99215674  0.          0.66143787  0.        ]
 [ 0.          1.          0.          0.44721359]
 [ 0.66143787  0.          0.99215674  0.        ]
 [ 0.          0.29408583  0.          0.98639399]]
